In [0]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [0]:
import sys

import warnings
warnings.filterwarnings("ignore")

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

import numpy as np
import pandas as pd
import time

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D

%matplotlib inline
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import io
import zipfile
from PIL import Image

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [65]:
!ls

adience_split.zip     mfs_model_5.h5	    trainHistoryDict
datalab		      mfs_train_full.h5     vggf_train_full.h5
inceptionHistoryDict  selfie_test_data	    vggHistoryDict
inception_model.h5    selfie_test_data.zip


### **Loading Datasets**

In [66]:
file_list = drive.ListFile({'q': "'1IsZnTUo2pSXDH00Xq42Ye8JJe768XUtB' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: adience_split_racial.zip, id: 1o1Bo8dG0jA7uwX_4QMpNe7avs4GRXfM8
title: selfie_test_data.zip, id: 1OIMpgpGwfW2Blyg7XGPQSIUbNYhrBvqH
title: adience_split.zip, id: 11GsvwXPQ4UGSSU2bILA_QeXXThHkFpy4
title: selfie_test_data, id: 1YcwE0YD_fFJxGvxPoQK7STEH58eGQXeZ
title: adience_split, id: 13HFhSYmArdZXIcf9jN78crzxFHl8Rxdt


In [0]:
data_downloaded = drive.CreateFile({'id': '11GsvwXPQ4UGSSU2bILA_QeXXThHkFpy4'})
data_downloaded.GetContentFile('adience_split.zip')

In [0]:
data = zipfile.ZipFile('adience_split.zip', 'r')
dir_list = data.namelist()

In [0]:
data.extractall('adience_split/')

In [9]:
!ls

adience_split  adience_split.zip  datalab


### Loading Models

In [50]:
file_list = drive.ListFile({'q': "'1AD8J5Me4tNE9qH_NgLEMUUqe6bpUDsOX' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: inceptionHistoryDict, id: 1kTirMRDEVzvC5OHpTpwO8s2pN7p0SMKd
title: inception_model.h5, id: 1eaHYc9rJbcSLqrnejZWxRnRDP0MZYgSj
title: vggHistoryDict, id: 1i5sSFNdNb35Z_F5dh4IJg2NwpPZlpye_
title: vggf_train_full.h5, id: 15uIxZUfNc6fAKwauoGN6doaAHhO31sfL
title: mfs_model_5.h5, id: 13CocAbYfPRo8AHHey0giJJ5GlL41Lg7d
title: mfs_model_2.h5, id: 1dxtpEBmwqR0rTLKd8cXxrO0SM7diLGY3
title: mfs_model.h5, id: 1IE2nVTq9VE-J0w1fr40V5DBoV3svF1Ps
title: trainHistoryDict, id: 1r-1jxxMwBqOd8gijmPEUXO2VWZIkgvTg
title: mfs_train_full.h5, id: 1x57aD18QE-mAjYo0mfjve8zBajKMLbuT


In [0]:
data_downloaded = drive.CreateFile({'id': '1x57aD18QE-mAjYo0mfjve8zBajKMLbuT'})
data_downloaded.GetContentFile('mfs_train_full.h5')
data_downloaded = drive.CreateFile({'id': '15uIxZUfNc6fAKwauoGN6doaAHhO31sfL'})
data_downloaded.GetContentFile('vggf_train_full.h5')
data_downloaded = drive.CreateFile({'id': '1eaHYc9rJbcSLqrnejZWxRnRDP0MZYgSj'})
data_downloaded.GetContentFile('inception_model.h5')

data_downloaded = drive.CreateFile({'id': '1r-1jxxMwBqOd8gijmPEUXO2VWZIkgvTg'})
data_downloaded.GetContentFile('trainHistoryDict')
data_downloaded = drive.CreateFile({'id': '1i5sSFNdNb35Z_F5dh4IJg2NwpPZlpye_'})
data_downloaded.GetContentFile('vggHistoryDict')
data_downloaded = drive.CreateFile({'id': '1kTirMRDEVzvC5OHpTpwO8s2pN7p0SMKd'})
data_downloaded.GetContentFile('inceptionHistoryDict')

In [0]:
data_downloaded = drive.CreateFile({'id': '13CocAbYfPRo8AHHey0giJJ5GlL41Lg7d'})
data_downloaded.GetContentFile('mfs_model_5.h5')

In [52]:
ls

adience_split.zip     inception_model.h5  selfie_test_data.zip  vggHistoryDict
datalab/              mfs_model_5.h5      trainHistoryDict
inceptionHistoryDict  mfs_train_full.h5   vggf_train_full.h5


### Loading Models

In [0]:
from keras.models import load_model

# Loading models
mfs_model = load_model('mfs_train_full.h5')
vggf_model = load_model('vggf_train_full.h5')
inc_model = load_model('inception_model.h5')
mfs_aug_model = load_model('mfs_model_5.h5')
# Loading histories

## 1. Loading Generators from Adience Dataset

In [15]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
source_path = 'adience_split/adience_split/'

train_generator = datagen.flow_from_directory(
        source_path+'train',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

val_generator = datagen.flow_from_directory(
        source_path+'val',
        target_size=(224, 224),
        batch_size=32, 
        shuffle = True)
#         class_mode='binary')

test_generator = datagen.flow_from_directory(
        source_path+'test',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

Found 8950 images belonging to 2 classes.
Found 3837 images belonging to 2 classes.
Found 5481 images belonging to 2 classes.


### Model Predictions

In [16]:
print(mfs_model.evaluate_generator(train_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(val_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(test_generator, pickle_safe=True))

[0.24186554320697679, 0.898547486046839]
[0.5582942061664103, 0.7836851707373493]
[0.579352014931726, 0.7775041051120616]


In [17]:
print(vggf_model.evaluate_generator(train_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(val_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(test_generator, pickle_safe=True))

[0.004001823546018513, 0.9985474860335195]
[0.34919430324292433, 0.9478759447485015]
[0.40860710303795544, 0.9366903849771218]


In [18]:
print(inc_model.evaluate_generator(train_generator, pickle_safe=True))
print(inc_model.evaluate_generator(val_generator, pickle_safe=True))
print(inc_model.evaluate_generator(test_generator, pickle_safe=True))

[0.7988634098441907, 0.7426815642191711]
[0.8258964402743357, 0.7281730519877084]
[0.812333361399128, 0.7356321839297956]


### Racial Predictions

In [0]:
data_downloaded = drive.CreateFile({'id': '1o1Bo8dG0jA7uwX_4QMpNe7avs4GRXfM8'})
data_downloaded.GetContentFile('adience_split_racial.zip')
data = zipfile.ZipFile('adience_split_racial.zip', 'r')
dir_list = data.namelist()
data.extractall('adience_split_racial/')

In [68]:
# Predicting on individual races
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
source_path = 'adience_split_racial/adience_split_racial/'

train_asian_generator = datagen.flow_from_directory(
        source_path+'train/asian',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

train_black_generator = datagen.flow_from_directory(
        source_path+'train/black',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

train_white_generator = datagen.flow_from_directory(
        source_path+'train/white',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')


val_asian_generator = datagen.flow_from_directory(
        source_path+'val/asian',
        target_size=(224, 224),
        batch_size=32, 
        shuffle = True)
#         class_mode='binary')

val_black_generator = datagen.flow_from_directory(
        source_path+'val/black',
        target_size=(224, 224),
        batch_size=32, 
        shuffle = True)
#         class_mode='binary')

val_white_generator = datagen.flow_from_directory(
        source_path+'val/white',
        target_size=(224, 224),
        batch_size=32, 
        shuffle = True)
#         class_mode='binary')



test_asian_generator = datagen.flow_from_directory(
        source_path+'test/asian',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

test_black_generator = datagen.flow_from_directory(
        source_path+'test/black',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

test_white_generator = datagen.flow_from_directory(
        source_path+'test/white',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

Found 2894 images belonging to 2 classes.
Found 308 images belonging to 2 classes.
Found 5748 images belonging to 2 classes.
Found 1226 images belonging to 2 classes.
Found 156 images belonging to 2 classes.
Found 2453 images belonging to 2 classes.
Found 1756 images belonging to 2 classes.
Found 231 images belonging to 2 classes.
Found 3494 images belonging to 2 classes.


In [69]:
print(mfs_model.evaluate_generator(train_asian_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(train_black_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(train_white_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(val_asian_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(val_black_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(val_white_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(test_asian_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(test_black_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(test_white_generator, pickle_safe=True))

[0.2116589171769459, 0.9162059433310297]
[0.3547160308082382, 0.8571428586910297]
[0.251027038975482, 0.8918754348924115]
[0.506289756317512, 0.8136215335393224]
[0.4975552872205392, 0.7692307722874177]
[0.5885181777388817, 0.7694659599274263]
[0.5277963373682765, 0.809794988746252]
[0.6163916340121975, 0.7597402592242022]
[0.6028138519732285, 0.7624499141044049]


In [70]:
print(vggf_model.evaluate_generator(train_asian_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(train_black_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(train_white_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(val_asian_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(val_black_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(val_white_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(test_asian_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(test_black_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(test_white_generator, pickle_safe=True))

[0.007448396226991828, 0.9972356599861782]
[0.0026718005424595207, 1.0]
[0.002337812565899863, 0.9991301322199025]
[0.3458031138134011, 0.9518760195758564]
[0.46536247432231903, 0.9358974389540844]
[0.34378261379320024, 0.9465960049405664]
[0.3199338390695298, 0.9481776765375854]
[0.5298297956134334, 0.8917748920329205]
[0.4451577029700133, 0.9338866628506011]


In [71]:
print(inc_model.evaluate_generator(train_asian_generator, pickle_safe=True))
print(inc_model.evaluate_generator(train_black_generator, pickle_safe=True))
print(inc_model.evaluate_generator(train_white_generator, pickle_safe=True))
print(inc_model.evaluate_generator(val_asian_generator, pickle_safe=True))
print(inc_model.evaluate_generator(val_black_generator, pickle_safe=True))
print(inc_model.evaluate_generator(val_white_generator, pickle_safe=True))
print(inc_model.evaluate_generator(test_asian_generator, pickle_safe=True))
print(inc_model.evaluate_generator(test_black_generator, pickle_safe=True))
print(inc_model.evaluate_generator(test_white_generator, pickle_safe=True))

[0.7058790524663312, 0.7601935038833513]
[0.5632022464430178, 0.8311688327170038]
[0.8583067671325195, 0.7291231731947048]
[0.6906014021613665, 0.7422512236854963]
[0.6139802978588984, 0.8076923122772803]
[0.9075681986344186, 0.7158581330685824]
[0.6909826848664425, 0.7699316628701595]
[0.6622871446919132, 0.7792207792207793]
[0.8832413417377264, 0.7155123068457955]


## 2. Loading selfie dataset

In [0]:
data_downloaded = drive.CreateFile({'id': '1OIMpgpGwfW2Blyg7XGPQSIUbNYhrBvqH'})
data_downloaded.GetContentFile('selfie_test_data.zip')
data = zipfile.ZipFile('selfie_test_data.zip', 'r')
dir_list = data.namelist()
data.extractall('selfie_test_data/')

In [58]:
ls 

adience_split.zip     mfs_model_5.h5        trainHistoryDict
datalab/              mfs_train_full.h5     vggf_train_full.h5
inceptionHistoryDict  selfie_test_data/     vggHistoryDict
inception_model.h5    selfie_test_data.zip


In [59]:
datagen = ImageDataGenerator(rescale=1./255)
source_path = 'selfie_test_data/selfie_test_data/'

asian_selfie_generator = datagen.flow_from_directory(
        source_path+'asian',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

black_selfie_generator = datagen.flow_from_directory(
        source_path+'black',
        target_size=(224, 224),
        batch_size=32, 
        shuffle = True)
#         class_mode='binary')

white_selfie_generator = datagen.flow_from_directory(
        source_path+'white',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

Found 2038 images belonging to 2 classes.
Found 1313 images belonging to 2 classes.
Found 15571 images belonging to 2 classes.


### Model/Racial Predictions

In [40]:
print(mfs_model.evaluate_generator(asian_selfie_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(black_selfie_generator, pickle_safe=True))
print(mfs_model.evaluate_generator(white_selfie_generator, pickle_safe=True))

[1.476551943602108, 0.42517173717253576]
[1.0831180027397371, 0.5407463823305407]
[1.3542446192500144, 0.43860381477815125]


In [41]:
print(vggf_model.evaluate_generator(asian_selfie_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(black_selfie_generator, pickle_safe=True))
print(vggf_model.evaluate_generator(white_selfie_generator, pickle_safe=True))

[3.259287856628897, 0.658979391267887]
[7.31332387873351, 0.3952779893373953]
[2.941449143096329, 0.6895510885697264]


In [42]:
print(inc_model.evaluate_generator(asian_selfie_generator, pickle_safe=True))
print(inc_model.evaluate_generator(black_selfie_generator, pickle_safe=True))
print(inc_model.evaluate_generator(white_selfie_generator, pickle_safe=True))

[0.915693784836347, 0.7482826300879339]
[4.234431565307245, 0.30616907844630614]
[0.8771154384730631, 0.7539657054666487]


In [60]:
print(mfs_aug_model.evaluate_generator(asian_selfie_generator, pickle_safe=True))
print(mfs_aug_model.evaluate_generator(black_selfie_generator, pickle_safe=True))
print(mfs_aug_model.evaluate_generator(white_selfie_generator, pickle_safe=True))

[0.9913691946755916, 0.43768400389617046]
[0.8922388112372509, 0.5620715917745621]
[1.0502387630358405, 0.40337807462016523]


In [43]:
ls

adience_split/        inception_model.h5    trainHistoryDict
adience_split.zip     mfs_train_full.h5     vggf_train_full.h5
datalab/              selfie_test_data/     vggHistoryDict
inceptionHistoryDict  selfie_test_data.zip


In [0]:
rm -rf selfie_test_data/

In [45]:
ls

adience_split/     inceptionHistoryDict  selfie_test_data.zip  vggHistoryDict
adience_split.zip  inception_model.h5    trainHistoryDict
datalab/           mfs_train_full.h5     vggf_train_full.h5


In [0]:
rm -rf adience_split/

In [47]:
ls

adience_split.zip     inception_model.h5    trainHistoryDict
datalab/              mfs_train_full.h5     vggf_train_full.h5
inceptionHistoryDict  selfie_test_data.zip  vggHistoryDict
